## Task

https://en.wikipedia.org/wiki/Sokoban#/media/File:Sokoban_ani.gif

Given map, boxes and destination points one must push all boxes to destination points.

### Solution idea
Optimal solution is not necessary. The main idea is to modify state space. Natural approach is to keep where is the storekeeper and all positions of boxes. However storekeeper moves when he is not pushing boxes are not so important and enlarge only state space. In this solution state remebers the position of boxes and enables to recreate possible box pushes (we remember leftmost and highest position reachable by storekeeper).

#### Heuristic function
We use A* algorithm so we need heuristic function.

Before executing algorithm we perform preprocessing. For every field we calculate how many pushes (not moves!) we have to make to place box on destination point.

Then in heuristic function we add all values for places where boxes are (we can add, not take max, because pushes are separate).

#### Recreating solution
A* returns only the sequenence of box pushes. We need to reconstruct storekeeper moves between pushes. We do that also with A* with manhattan distance as heuristic function.

### Input
```
WWWWWWWWW
WWW..WWWW
W.......W
WKB***G.W
W.......W
WWWWWWWWW
```
`W` wall

`K` storekeeper

`B` box

`*` box on destination point

`G` destination point
### Output
Sequence of storekeeper moves to finish the riddle.

`U` - up

`L` - left

`R` - right

`D` - down

## Imports

In [1]:
import numpy as np
import queue
from dataclasses import dataclass, field
from typing import Any

## Code

In [2]:
# useful constants

map_char_to_board = {
    'W': 1, # wall
    'K': 0, # storekeeper
    'B': 0, # box
    'G': 2, # target
    '*': 2, # box on target
    '+': 2, # storekeeper on target
    '.': 0  # empty place
}

moves_pairs = [(1,0),(-1,0),(0,1),(0,-1)]

possible_directions = ['U','D','L','R']

coordinates_to_moves = {
    (-1,0): 'U',
    (1,0): 'D',
    (0,1): 'R',
    (0,-1): 'L'
}

moves_to_coordinates = {
    'U': (-1,0),
    'D': (1,0),
    'L': (0,-1),
    'R': (0,1)
}


@dataclass(order=True)
class PrioritizedItem:
    priority: int
    item: Any=field(compare=False)
        
class Element(PrioritizedItem):
    
    def __init__(self,priorority,item):
        self.priority = priorority
        self.item = item

In [3]:
# data loading
def read_data(input_file='zad_input.txt'):
    boxes = set()
    destination_points = set()

    data = open(input_file, 'r')
    data = data.readlines()
    data = list(map(lambda x: x.strip(), data))
    height = len(data)
    width = len(data[0])
    board = np.zeros([height,width], dtype = np.int8)
    for i in range(height):
        for j in range(width):
            character = data[i][j]
            board[i,j] = map_char_to_board[character]
            if character=='K' or character=='+':
                storekeeper = (i,j)
            if character=='B' or character=='*':
                boxes.add((i,j))
            if character=='G' or character=='*' or character=='+':
                destination_points.add((i,j))
    return boxes,destination_points,board,storekeeper

In [4]:
def create_distances(board,destination_points):
    distances = np.full(board.shape,10**4,dtype=np.int16)
    q = queue.Queue()
    visited = np.zeros(board.shape,dtype=np.int8)
    for i,j in destination_points:
        q.put((i,j))
        distances[i,j]=0
        visited[i,j]=1
    while not q.empty():
        i0,j0 = q.get()
        for i,j in moves_pairs:
            if not visited[i0+i,j0+j] and board[i0+i,j0+j]!=1 and board[i0+2*i,j0+2*j]!=1:
                distances[i0+i,j0+j] = distances[i0,j0]+1
                q.put((i0+i,j0+j))
                visited[i0+i,j0+j] = 1
    return distances
                
def heuristic(distances,boxes):
    sum_ = 0
    for i,j in boxes:
        sum_ += distances[i,j]
    return sum_

In [5]:
def a_star(board,storekeeper,boxes,destination_points):
    distances = create_distances(board,destination_points)
    states = set()
    q = queue.PriorityQueue()
    storekeeper,availible_moves = update_moves(board,storekeeper,boxes)
    q.put(Element(0,(0,storekeeper,boxes,availible_moves,None)))
    while not q.empty():
        elem = q.get()
        cost_of_getting,storekeeper,boxes,availible_moves,father = elem.item
        if boxes==destination_points:
            return father
        state = create_state(storekeeper,boxes)
        if state not in states:
            states.add(state)
            for move in availible_moves:
                new_boxes = move_box(move,boxes)
                storekeeper,new_availible_moves = update_moves(board,move[0],new_boxes)
                q.put(Element(cost_of_getting+1+heuristic(distances,new_boxes),
                                      (cost_of_getting+1,storekeeper,new_boxes,new_availible_moves,
                                       {'move': move, 'father': father})))
    return None

def create_state(storekeeper,boxes):
    list_boxes=tuple(sorted(list(boxes)))
    return (storekeeper,list_boxes)

In [6]:
def update_moves(board,storekeeper,boxes):
    q = queue.Queue()
    visited = np.zeros(board.shape,dtype=np.int8)
    q.put(storekeeper)
    visited[storekeeper] = 1
    vert,hor = board.shape
    availible_moves = []
    while not q.empty():
        i0,j0 = q.get()
        for i,j in moves_pairs:
            if i0<vert or (i0==vert and j0<hor):
                vert,hor = i0,j0
            if (i0+i,j0+j) in boxes:
                if board[i0+2*i,j0+2*j]!=1 and (i0+2*i,j0+2*j) not in boxes:
                    availible_moves.append(((i0+i,j0+j),(i,j)))
            else:
                if not visited[i0+i,j0+j] and board[i0+i,j0+j]!=1:
                    visited[i0+i,j0+j] = 1
                    q.put((i0+i,j0+j))
    return (vert,hor),availible_moves

def move_box(move,boxes):
    b_i,b_j = move[0]
    i,j = move[1]
    new_boxes = boxes.copy()
    new_boxes.remove((b_i,b_j))
    new_boxes.add((b_i+i,b_j+j))
    return new_boxes

def solved(destination_points,boxes):
    return destination_points==boxes

In [7]:
def replay_route(storekeeper,board,dictionary,boxes):
    
    def reverse_dicitonary(dictionary,l,key='move'):
        if dictionary is None:
            return list(reversed(l))
        l.append(dictionary[key])
        return reverse_dicitonary(dictionary['father'],l,key=key)
    
    def manhattan_dist(a,b):
        return abs(a[0]-b[0])+abs(a[1]-b[1])
    
    def a_star(storekeeper,destination,boxes):
        states = set()
        q = queue.PriorityQueue()
        q.put(Element(0,(0,storekeeper,None)))
        while not q.empty():
            elem = q.get()
            cost_of_getting,storekeeper,father = elem.item
            if storekeeper==destination:
                return reverse_dicitonary(father,[],key='letter')
            if storekeeper not in states:
                states.add(storekeeper)
                for i,j in moves_pairs:
                    new_storekeeper = (storekeeper[0]+i,storekeeper[1]+j)
                    if board[new_storekeeper]!=1 and new_storekeeper not in boxes:
                        q.put(Element(cost_of_getting+1+manhattan_dist(destination,new_storekeeper),
                                          (cost_of_getting+1,new_storekeeper,
                                           {'letter': coordinates_to_moves[(i,j)], 'father': father})))
        return None
        
    list_of_moves = reverse_dicitonary(dictionary,[])
    moves = []
    for box,direction in list_of_moves:
        moves.extend(a_star(storekeeper,(box[0]-direction[0],box[1]-direction[1]),boxes))
        storekeeper = box
        moves.append(coordinates_to_moves[direction])
        boxes.remove(box)
        boxes.add((box[0]+direction[0],box[1]+direction[1]))
    return moves

In [8]:
def prog(input_file='zad_input.txt'):
    boxes,destination_points,board,storekeeper = read_data(input_file=input_file)
    dictionary = a_star(board,storekeeper,boxes,destination_points)
    return replay_route(storekeeper,board,dictionary,boxes)

In [9]:
print(prog())

['D', 'R', 'R', 'R', 'U', 'R', 'L', 'D', 'L', 'L', 'L', 'U', 'U', 'R', 'R', 'U', 'R', 'D', 'R', 'D', 'D', 'L', 'L', 'U', 'D', 'L', 'L', 'U', 'U', 'R', 'R', 'D', 'R', 'L', 'U', 'L', 'L', 'D', 'R', 'U', 'R', 'U', 'R', 'D']
